In [445]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Modeling Helpers
from sklearn.cross_validation import train_test_split

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualizations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

In [446]:
housing_df = pd.read_csv('housing_train.csv')

In [447]:
#get an idea of what the data looks like
housing_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [448]:
#Look for possible nulls and columns that are sparse
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [449]:
#start making a cleaned up version of the data
housing_df_clean = housing_df.copy()

In [450]:
#drop all columns that have few values
del housing_df_clean['Alley']
del housing_df_clean['FireplaceQu']
del housing_df_clean['PoolQC']
del housing_df_clean['Fence']
del housing_df_clean['MiscFeature']

In [451]:
#make mapping for dummy categories
generic_equiv = {"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA":-999}

In [452]:
#every column here is categorical and will be changed using one-hot encoding and then concatenated onto the original data
dummy_categories = []

dummy_categories.append('LotShape')
dummy_categories.append('MSZoning')
dummy_categories.append('Street')
dummy_categories.append('LandContour')
dummy_categories.append('Utilities')
dummy_categories.append('LotConfig')
dummy_categories.append('LandSlope')
dummy_categories.append('Neighborhood')
dummy_categories.append('Condition1')
dummy_categories.append('Condition2')
dummy_categories.append('BldgType')
dummy_categories.append('HouseStyle')
dummy_categories.append('RoofStyle')
dummy_categories.append('RoofMatl')
dummy_categories.append('Exterior1st')
dummy_categories.append('Exterior2nd')
dummy_categories.append('MasVnrType')
dummy_categories.append('Foundation')
dummy_categories.append('ExterQual')
dummy_categories.append('ExterCond')
dummy_categories.append('BsmtQual')
dummy_categories.append('BsmtCond')
dummy_categories.append('BsmtExposure')
dummy_categories.append('BsmtFinType1')
dummy_categories.append('BsmtFinType2')
dummy_categories.append('Heating')
dummy_categories.append('HeatingQC')
dummy_categories.append('CentralAir')
dummy_categories.append('Electrical')
dummy_categories.append('KitchenQual')
dummy_categories.append('Functional')
dummy_categories.append('GarageType')
dummy_categories.append('GarageFinish')
dummy_categories.append('GarageQual')
dummy_categories.append('GarageCond')
dummy_categories.append('PavedDrive')
dummy_categories.append('SaleType')
dummy_categories.append('SaleCondition')
       
extra_frames = []
for category in dummy_categories:    
    data = pd.get_dummies(prefix=category,data=housing_df_clean[category])
    frames = [housing_df_clean,data]
    housing_df_clean = pd.concat(frames,axis=1)
    del housing_df_clean[category]

In [453]:
#put SalePrice(our target variable) back at the end of the data
housing_df_clean['FinalPrice'] = housing_df_clean['SalePrice']
del housing_df_clean['SalePrice']
housing_df_clean['SalePrice'] = housing_df_clean['FinalPrice']
del housing_df_clean['FinalPrice']
housing_df_clean.head()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,250000


In [454]:
#any remaining null values need to be replaced by zero
housing_df_clean = housing_df_clean.fillna(0)

In [455]:
housing_df_clean.isnull().any()

Id                       False
MSSubClass               False
LotFrontage              False
LotArea                  False
OverallQual              False
OverallCond              False
YearBuilt                False
YearRemodAdd             False
MasVnrArea               False
BsmtFinSF1               False
BsmtFinSF2               False
BsmtUnfSF                False
TotalBsmtSF              False
1stFlrSF                 False
2ndFlrSF                 False
LowQualFinSF             False
GrLivArea                False
BsmtFullBath             False
BsmtHalfBath             False
FullBath                 False
HalfBath                 False
BedroomAbvGr             False
KitchenAbvGr             False
TotRmsAbvGrd             False
Fireplaces               False
GarageYrBlt              False
GarageCars               False
GarageArea               False
WoodDeckSF               False
OpenPorchSF              False
                         ...  
GarageFinish_Unf         False
GarageQu

In [456]:
#check that there are no null values left
housing_df_clean.isnull().any().any()

False

In [457]:
#Look at the values that are highly correlated to SalePrice.  First experiment with choosing a subsection
#of only the highly correlated values to use for the model led to worse results. 
housing_corr = housing_df_clean.corr()
housing_corr['SalePrice'].sort_values(ascending=False)

SalePrice                1.000000
OverallQual              0.790982
GrLivArea                0.708624
GarageCars               0.640409
GarageArea               0.623431
TotalBsmtSF              0.613581
1stFlrSF                 0.605852
FullBath                 0.560664
BsmtQual_Ex              0.553105
TotRmsAbvGrd             0.533723
YearBuilt                0.522897
YearRemodAdd             0.507101
KitchenQual_Ex           0.504094
Foundation_PConc         0.497734
MasVnrArea               0.472614
Fireplaces               0.466929
ExterQual_Gd             0.452466
ExterQual_Ex             0.451164
BsmtFinType1_GLQ         0.434597
HeatingQC_Ex             0.434543
GarageFinish_Fin         0.419678
Neighborhood_NridgHt     0.402149
BsmtFinSF1               0.386420
SaleType_New             0.357509
SaleCondition_Partial    0.352060
GarageType_Attchd        0.335961
MasVnrType_Stone         0.330476
Neighborhood_NoRidge     0.330424
WoodDeckSF               0.324413
KitchenQual_Gd

In [458]:
#another experiment in the cell below using only the interquartile range (mid 50%) led to worse results 
housing_df_clean.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,0.083562,0.002055,0.867808,0.069178,0.002740,0.008219,0.013699,0.820548,0.085616,180921.195890
std,421.610009,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,0.276824,0.045299,0.338815,0.253844,0.052289,0.090317,0.116277,0.383862,0.279893,79442.502883
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34900.000000
25%,365.750000,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,129975.000000
50%,730.500000,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,163000.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,755000.000000


In [459]:
# housing_df_clean = housing_df_clean[(housing_df_clean['SalePrice'] > housing_df_clean['SalePrice'].quantile(.05)) & (housing_df_clean['SalePrice'] < housing_df_clean['SalePrice'].quantile(.95)) ]

In [460]:
x = housing_df_clean.copy()
del x['SalePrice']
y = housing_df_clean['SalePrice']


In [461]:
#split my data, reserving 20 percent for testing purposes
validation_size = 0.2
seed = 7
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=validation_size, random_state=seed)

In [462]:
#find the best alpha for Ridge regression the hard way
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
alphas = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]

for alpha in alphas:    
    ridge_cv = Ridge(alpha)
    scores = cross_val_score(ridge_cv, x, y, cv=10)
    print("Alpha = " + str(alpha) + "\n" +"Score = "+ str(scores.mean()) + "\n")

Alpha = 0.0001
Score = 0.819247509145

Alpha = 0.001
Score = 0.81938958526

Alpha = 0.01
Score = 0.820730144195

Alpha = 0.1
Score = 0.829472188269

Alpha = 1
Score = 0.844120054894

Alpha = 10
Score = 0.849575634947

Alpha = 100
Score = 0.84297983176

Alpha = 1000
Score = 0.805977602578

Alpha = 10000
Score = 0.755247446872



In [463]:
#find the best alpha for Ridge regression the easy way
from sklearn import linear_model
reg = linear_model.RidgeCV(alphas=[0.0001,0.001,0.01,0.1,1,10,100,1000,10000])
reg.fit(x,y)       


print "Best alpha choice is: " + str(reg.alpha_)

Best alpha choice is: 10.0


In [464]:
#actual score using the parameters found above
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=10).fit(x_train,y_train)
print("training set score: %f" % ridge.score(x_train, y_train))
print("test set score: %f" % ridge.score(x_test, y_test))

training set score: 0.896557
test set score: 0.880309


In [465]:
#experiment with ElasticNet, which is another form of regression that uses an alpha and an L1 ratio.  This led to worse results.
reg = linear_model.ElasticNetCV(alphas=[0.0001,0.001,0.01,0.1,1,10,100,1000,10000],l1_ratio=[.1, .5, .7, .9, .95, .99, 1])
reg.fit(x,y)       


print "Best alpha choice is: " + str(reg.alpha_)
print "Best l1_ratio choice is: " + str(reg.l1_ratio_)

Best alpha choice is: 100.0
Best l1_ratio choice is: 1.0


In [466]:
elastic = linear_model.ElasticNet(alpha=100,l1_ratio=1).fit(x_train,y_train)
print("training set score: %f" % elastic.score(x_train, y_train))
print("test set score: %f" % elastic.score(x_test, y_test))

training set score: 0.923941
test set score: 0.874030


In [471]:
y_test.count()

292

In [472]:
#a look at the predicted values and actual values
from __future__ import division

for i in range(1,292,2): 
    predicted_value = ridge.predict(x_test)[i]
    actual_value = y_test.iloc[i]
    percent_difference = (predicted_value - actual_value)/predicted_value
    
    print("predicted: " + str(predicted_value))
    print("actual: " + str(actual_value))
    print("difference: " + str( predicted_value - actual_value ) )
    print("percent difference: " + str(abs(percent_difference * 100)))
    print("\n")

predicted: 204023.944795
actual: 189000
difference: 15023.9447946
percent difference: 7.36381448253


predicted: 93082.7459529
actual: 60000
difference: 33082.7459529
percent difference: 35.5412226124


predicted: 336575.926697
actual: 318000
difference: 18575.9266972
percent difference: 5.51908952


predicted: 131411.860928
actual: 124000
difference: 7411.86092831
percent difference: 5.64017652285


predicted: 112557.735539
actual: 129500
difference: -16942.2644613
percent difference: 15.05206584


predicted: 166193.466776
actual: 177500
difference: -11306.5332244
percent difference: 6.80323567693


predicted: 198435.100293
actual: 155835
difference: 42600.1002925
percent difference: 21.4680266897


predicted: 179282.238519
actual: 178000
difference: 1282.23851909
percent difference: 0.715206665024


predicted: 144847.559202
actual: 152000
difference: -7152.44079833
percent difference: 4.93790909405


predicted: 181451.391382
actual: 170000
difference: 11451.391382
percent difference: